In [2]:
import torch, os
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

CUDA available: True
Wed Oct 29 21:45:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.57                 Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8             14W /   65W |     892MiB /   4096MiB |      4%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [4]:
import accelerate
import transformers
print(accelerate.__version__)
print(transformers.__version__)
import torch
print("torch:", torch.__version__)
print("cuda runtime visible to torch:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
import bitsandbytes as bnb
print(bnb.__version__)

1.11.0
4.57.1
torch: 2.7.1+cu118
cuda runtime visible to torch: 11.8
cuda available: True
0.48.1


In [ ]:
import json
import numpy as np
import faiss
import time
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

class RAGService:
    def __init__(self):
        print("Đang khởi tạo hệ thống RAG tiếng Việt trên GPU...\n")
        start_time = time.time()

        # 1. Mô hình embed
        self.embed_model = SentenceTransformer('all-MiniLM-L6-v2')

        # 2. FAISS index
        try:
            self.index = faiss.read_index('../data/faiss_index.bin')
        except Exception as e:
            raise RuntimeError(f"Không thể load FAISS index: {e}")

        # 3. Dữ liệu chunks
        try:
            with open('../data/faiss_index.meta.json', 'r', encoding='utf-8') as f:
                self.chunks = json.load(f)
        except Exception as e:
            raise RuntimeError(f"Không thể load FAISS metadata: {e}")

        # 4. Mô hình sinh tiếng Việt nhỏ, load GPU
        model_name = "NlpHUST/gpt-neo-vi-small"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.gen_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",   # tự động phân bổ GPU/CPU
            low_cpu_mem_usage=True
        )

        # Kiểm tra GPU
        self.device = 0 if torch.cuda.is_available() else -1
        device_name = "GPU" if self.device == 0 else "CPU"
        print(f">>> Sẽ sử dụng thiết bị: {device_name}")

        self.generator = pipeline(
            "text-generation",
            model=self.gen_model,
            tokenizer=self.tokenizer
        )

        print(f"Khởi tạo hoàn tất trong {time.time() - start_time:.2f}s\n")

    def retrieve(self, query, top_k=5):
        t0 = time.time()
        try:
            query_emb = self.embed_model.encode([query])
            distances, indices = self.index.search(query_emb, top_k)
            results = []
            for i, idx in enumerate(indices[0]):
                chunk = self.chunks[idx]
                score = 1 - distances[0][i] / np.max(distances)
                results.append({'chunk': chunk, 'score': score})
            results.sort(key=lambda x: x['score'], reverse=True)
            print(f">>> Truy xuất hoàn tất trong {time.time() - t0:.2f}s")
            return results
        except Exception as e:
            print("Lỗi khi truy xuất:", e)
            return []

    def generate(self, query, retrieved):
        if not retrieved:
            return {'answer': "Không tìm thấy thông tin phù hợp.", 'sources': []}

        t0 = time.time()
        context = ' '.join([r['chunk']['content'] for r in retrieved])
        prompt = (
            f"Câu hỏi: {query}\n"
            f"Ngữ cảnh: {context}\n\n"
            "Trả lời ngắn gọn, chính xác và tự nhiên bằng tiếng Việt:"
        )

        try:
            response = self.generator(
                prompt,
                max_length=150,  # giảm để an toàn GPU
                do_sample=True,
                temperature=0.7,
                num_return_sequences=1
            )[0]['generated_text']
            print(f">>> Sinh câu trả lời trong {time.time() - t0:.2f}s")
            return {'answer': response, 'sources': [r['chunk']['metadata'] for r in retrieved]}
        except RuntimeError as e:
            # Nếu GPU OOM, fallback sang CPU
            if "out of memory" in str(e):
                torch.cuda.empty_cache()
                print("GPU OOM! Chuyển sang CPU tạm thời...")
                self.generator.device = -1
                response = self.generator(
                    prompt,
                    max_length=120,
                    do_sample=True,
                    temperature=0.7,
                    num_return_sequences=1
                )[0]['generated_text']
                return {'answer': response, 'sources': [r['chunk']['metadata'] for r in retrieved]}
            else:
                print("Lỗi khi sinh câu trả lời:", e)
                return {'answer': "Không thể sinh câu trả lời.", 'sources': []}


def main():
    rag = RAGService()
    while True:
        query = input("\nNhập câu hỏi (hoặc gõ 'exit' để thoát): ").strip()
        if query.lower() == 'exit':
            break
        retrieved = rag.retrieve(query)
        result = rag.generate(query, retrieved)

        print("\n===== KẾT QUẢ =====")
        print("Trả lời:", result['answer'])
        if result['sources']:
            print("\nNguồn tham chiếu:")
            for src in result['sources']:
                print("-", src)
        print("\n====================================================")

if __name__ == "__main__":
    main()

d:\RAG\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Đang khởi tạo hệ thống RAG tiếng Việt trên GPU...



Device set to use cuda:0


>>> Sẽ sử dụng thiết bị: GPU
Khởi tạo hoàn tất trong 8.37s



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

>>> Truy xuất hoàn tất trong 0.02s
>>> Sinh câu trả lời trong 4.55s

===== KẾT QUẢ =====
Trả lời: Câu hỏi: phố đi bộ nguyễn huệ ở đâu?
Ngữ cảnh: bỏ hoang phế. Năm 1802, Gia Long định đô ở Huế, thì Thăng Long là thành trì của Tổng trấn bên nhau khi đi chợ, đi nương, thăm hỏi. Phổ biến có tục cướp vợ. DÂN H MÔNG [Paraphrased] : nơi ngồi nghỉ của Nguyễn Huệ và Nguyễn Nhạc, sau mỗi đợt luyện quân ; Vườn mít - cánh đồng Cô Hầu ... (Cite page 460) [Paraphrased] Thăng Long, Nguyễn Nhạc sợ không kiềm chế được Nguyễn Huệ, bèn đem quân ra Bắc Hà. _ Nghe tin vua Tâ... (Cite page 62) [Paraphrased] cầm bó hương đi trước ( ông già này phải hiền lành, ăn ở đạo đức, vợ chồng song toàn, lắm con nhiều ... (Cite page 243)

Trả lời ngắn gọn, chính xác và tự nhiên bằng tiếng Việt: sĩ Vũ Đình Du.
Tôi đã viết mấy câu về H'Mamor, kể cho mẹ nghe. Mẹ nuôi tôi, chỉ lấy tôi làm con dâu, mẹ tôi là mẹ ruột. Mẹ tôi đã nuôi tôi đến ngày trưởng thành, thì mẹ tôi cũng là mẹ ruột của tôi.
Mẹ tôi hay ốm, tôi cũng hay ốm,

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> Truy xuất hoàn tất trong 0.02s
>>> Sinh câu trả lời trong 1.76s

===== KẾT QUẢ =====
Trả lời: Câu hỏi: năm 1945 ở việt nam có sự kiện gì nổi bật?
Ngữ cảnh: gia kỹ sư hai nước Australia và Việt Nam, được đưa vào sử dụng ngày [Paraphrased] 236 CÁC DÂN NAM Vào những năm đầu thập niên 1950, khi cuộc kháng chiến của nhân dân Việt Nam giành đ... (Cite page 235) [Paraphrased] 80 CÁC DÂN NAM Sau cách mạng tháng Tám năm 1945 đến nay, nhờ chính sách nhất quán của Đảng và Nhà nư... (Cite page 79) [Paraphrased] ##c và súng kíp, nhưng chỉ sau 3 ngày thành lập, ngày 25 - 12 - 1944, Đội Việt Nam Tuyên truyền Giải... (Cite page 388) [Paraphrased] 868 CÁC & THÀNH PHỐ Nhà tù Chợ Chu Do thực dân Pháp xây dựng vào năm 1916 để giam cầm những người yê... (Cite page 867)

Trả lời ngắn gọn, chính xác và tự nhiên bằng tiếng Việt: đối ngoại của Nam ta hiện nay là tập trung chú trọng củng cố vững chắc khu vực Đông Nam Á.
Từ khi Nam ta dần ổn định chính sách đối ngoại, tiếp tục củng cố quan hệ với Đông Nam Á,

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> Truy xuất hoàn tất trong 0.02s
>>> Sinh câu trả lời trong 4.91s

===== KẾT QUẢ =====
Trả lời: Câu hỏi: xin chào
Ngữ cảnh: [Paraphrased] cư mới, nhiều nhà chung cư cao tầng, nhiều đô thị mới, khu công nghiệp, nhiều làng xóm được quy hoạc... (Cite page 196) [Paraphrased] 942 CÁC & THÀNH PHỐ Chùa dựng theo kiểu chữ tam ( 三 ), ba gian nối liền nhau, gồm chính điện, hậu tổ... (Cite page 941) [Paraphrased] 622 CÁC & THÀNH PHỐ sử chống giặc ngoại xâm cuối thế kỷ XIX, đầu thế kỷ XX và cuộc biểu tình đấu tra... (Cite page 621) [Paraphrased] 480 CÁC & THÀNH PHỐ Đa, Thanh Xuân, Hoàng Mai, Cầu Giấy, Tây Hồ, Long Biên và 5 huyện : Gia Lâm, Đôn... (Cite page 479) [Paraphrased] trong sự nghiệp chống giặc ngoại xâm. Có thể nói mỗi tấc đất, mỗi địa danh Xứ Lạng đều thấm đẫm chiế... (Cite page 634)

Trả lời ngắn gọn, chính xác và tự nhiên bằng tiếng Việt:aphrased] cany dascany dascany dascany dascany tucany dascanyt dascany dascany tucany dascany dascany/tascany dascany dascany dascany dascany dasca